# FAHASA

## Khởi Tạo

In [1]:
from urllib.request import urlopen
import urllib.request as urllib2
from urllib.error import HTTPError
from bs4 import BeautifulSoup, SoupStrainer
import numpy as np
import pandas as pd
import concurrent.futures
from multiprocessing import Pool
import requests
from selenium import webdriver
from selenium.common.exceptions import *
import matplotlib.pyplot as plt
import seaborn as sns
from selenium.webdriver.common.by import By
import argparse
from datetime import datetime
import os
import re
import time
import argparse
from collections import Counter
from datetime import datetime
import os
import regex as re
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException, ElementNotVisibleException, StaleElementReferenceException
from selenium.webdriver.support.ui import Select
from urllib.request import urlopen
from urllib.error import HTTPError
from selenium.webdriver.common.by import By
import pandas as pd
import geckodriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager

## MAIN

In [6]:
pages = []
goc='https://www.fahasa.com/sach-trong-nuoc/van-hoc-trong-nuoc/tieu-thuyet.html' 
pages.append(goc)
def get_links(trang):
    for i in range(2,trang+1):
        pages.append('https://www.fahasa.com/sach-trong-nuoc/van-hoc-trong-nuoc/tieu-thuyet.html?order=num_orders&limit=24&p='+str(i))
get_links(86)

In [7]:
lon = []
img = []
loun = []
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}

def take_name_and_img(url):
    r = requests.get(url,headers=headers)
    r.raise_for_status()
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find_all('span', class_ ='product-image')
    for ptd in s:
        tam = ptd.find('img')
        lon.append(tam['alt'])
        img.append(tam['data-src'])
    s = soup.find_all('a', class_='product-image')
    for ptd in s:
        loun.append(ptd['href'])
    
for url in pages:
    take_name_and_img(url)

In [ ]:
lop = []
loa = []
genre = []

def take_prices_inf_genre(url):
    r = requests.get(url,headers=headers)
    r.raise_for_status()
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('span',class_='price')
    try:
        lop.append(s.text)
        s = soup.find('div',class_='std')
        loa.append(s.text)
        s = soup.find('ol',class_='breadcrumb').find_all('a')
        temp = []
        for ptd in s:
            temp.append(ptd.text)
        genre.append(temp)
    except:
        lon.pop(len(lop))
        img.pop(len(lop))
    

for url in loun:
    take_prices_inf_genre(url)

In [13]:
dict_pro={"Ten ": lon,"Gia ": lop ,"Thong tin": loa,"Anh":img,"The loai":genre}
df=pd.DataFrame(dict_pro)

In [14]:
df.to_csv("lgh.csv")

## User

In [34]:
import re

In [ ]:
lonn = []
def no_accent_vietnamese(s):
    s = re.sub('[áàảãạăắằẳẵặâấầẩẫậ]', 'a', s)
    s = re.sub('[ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬ]', 'A', s)
    s = re.sub('[éèẻẽẹêếềểễệ]', 'e', s)
    s = re.sub('[ÉÈẺẼẸÊẾỀỂỄỆ]', 'E', s)
    s = re.sub('[óòỏõọôốồổỗộơớờởỡợ]', 'o', s)
    s = re.sub('[ÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢ]', 'O', s)
    s = re.sub('[íìỉĩị]', 'i', s)
    s = re.sub('[ÍÌỈĨỊ]', 'I', s)
    s = re.sub('[úùủũụưứừửữự]', 'u', s)
    s = re.sub('[ÚÙỦŨỤƯỨỪỬỮỰ]', 'U', s)
    s = re.sub('[ýỳỷỹỵ]', 'y', s)
    s = re.sub('[ÝỲỶỸỴ]', 'Y', s)
    s = re.sub('đ', 'd', s)
    s = re.sub('Đ', 'D', s)
    return s

def tach(vcl):
    for i in range(len(vcl)):
        if vcl[i] == '(' or vcl[i]=='-':
            return vcl[:i]
    return vcl
goc0 = 'https://www.goodreads.com/'
goc1 = 'https://www.goodreads.com/search?q='
newlist = []

for ptd in lon:
    lonn.append(tach(ptd))
for lgh in lonn:
    url = goc1
    for ptd in lgh:
        if ptd == ' ':
            url += '%20'
        else:
            url+= ptd

    url = no_accent_vietnamese(url)
    try:
        html=urlopen(url) 
        bs = BeautifulSoup(html.read(), 'html.parser') 
        ptd = bs.find_all('a',class_='bookTitle')
        newlist.append(goc0+ptd[0]['href'])
    except:
        newlist.append('')


In [99]:
inf = []
rating = []
all_genres = []
def genre_about(url):
    try: 
        html=urlopen(url) 
        bs = BeautifulSoup(html.read(), 'html.parser') 
        dtb = re.findall(r'renderRatingGraph\([\s]*\[[0-9,\s]+', str(bs))[0]
        dtb = ' '.join(dtb.split())
        dtb = [int(c.strip()) for c in dtb.split('[')[1].split(',')]
        dtb_dict = {'5': dtb[0],'4': dtb[1],'3': dtb[2],'2': dtb[3],'1':  dtb[4]}
        genres = []
        for node in bs.find_all('div', {'class': 'left'}):
            current_genres = node.find_all('a', {'class': 'actionLinkLite bookPageGenreLink'})
            current_genre = ' > '.join([g.text for g in current_genres])
            if current_genre.strip():
                genres.append(current_genre)
        ptd = bs.find('div',id="description")
        inf.append(ptd.text)
        rating.append(dtb_dict)
        all_genres.append(genres)
    except:
        inf.append('')
        rating.append('')
        all_genres.append('')

for ptd in newlist:
    genre_about(ptd)

In [ ]:
user_id = []
user_rating = []
user_rating_book = []
options = webdriver.FirefoxOptions()
options.headless = True 
def get_rating1(driver,book):
    soup = driver.page_source
    source =BeautifulSoup(soup, 'html.parser')
    for ptd in source.find_all('a',class_='left imgcol'):
        user_rating_book.append(book)
        user_id.append(ptd['href'])
    for ptd in source.find_all('div',class_='reviewHeader uitext stacked'):
        if ptd.find_all('span',class_='staticStars notranslate')== []:
            user_rating.append('0')
        else:
            user_rating.append(ptd.find('span',class_='staticStars notranslate').text)
    return 

def main(url,book): 
    try:
        driver = webdriver.Firefox(executable_path='firefoxdriver', options=options)
        driver.get(url)
        kt = True
        select = Select(driver.find_element(By.NAME, str('language_code')))
        select.select_by_value('en')
        time.sleep(4)
        old = len(user_id)
        get_rating1(driver,book)
        if (old == len(user_id)) :
            kt = False
            time.sleep(10)
        page_counter = 1
        while (True):
            try:
                if driver.find_element(By.XPATH, "//a[@class='next_page']"):
                    driver.find_element(By.XPATH, "//a[@class='next_page']").click()
                    time.sleep(3)
                    get_rating1(driver,book)
            except :
                os.system("taskkill /im firefox.exe /f")
                return 
    except:
        user_rating_book.append('')
        user_id.append('')
        user_rating.append('')
        os.system("taskkill /im firefox.exe /f")
                
for i in range(len(newlist)):
    main(newlist[i],lon[i])

In [13]:
dict_pro={"id ": user_id,"rating ": user_rating ,"name": user_rating_book}
df=pd.DataFrame(dict_pro)

In [14]:
df.to_csv("lgh6.csv")